# "Foundations of Computer Science" course (F9101Q001)
## Final Project

**Matteo Bressan - 765957**


---

The current project refers to [this provided guideline](http://gianluca.dellavedova.org/foundationsCS/2019-project)

####0.   **Common part - Libraries, configurations and files import**


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from calendar import isleap

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
loans_lenders_url = '/content/drive/My Drive/additional-kiva-snapshot/loans_lenders.csv'
loans_lenders_import = pd.read_csv(loans_lenders_url)
loans_lenders_import.dtypes

loan_id     int64
lenders    object
dtype: object

In [ ]:
loans_lenders_import.head(2)

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."


In [ ]:
loans_url = '/content/drive/My Drive/additional-kiva-snapshot/loans.csv'
loans_import = pd.read_csv(loans_url)
loans_import.dtypes

loan_id                              int64
loan_name                           object
original_language                   object
description                         object
description_translated              object
funded_amount                      float64
loan_amount                        float64
status                              object
activity_name                       object
sector_name                         object
loan_use                            object
country_code                        object
country_name                        object
town_name                           object
currency_policy                     object
currency_exchange_coverage_rate    float64
currency                            object
partner_id                         float64
posted_time                         object
planned_expiration_time             object
disburse_time                       object
raised_time                         object
lender_term                        float64
num_lenders

In [ ]:
loans_import.head(2)

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner


In [ ]:
lenders_url = '/content/drive/My Drive/additional-kiva-snapshot/lenders.csv'
lenders_import = pd.read_csv(lenders_url)
lenders_import.dtypes

permanent_name        object
display_name          object
city                  object
state                 object
country_code          object
member_since           int64
occupation            object
loan_because          object
loan_purchase_num    float64
invited_by            object
num_invited            int64
dtype: object

In [ ]:
lenders_import.head(5)

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [ ]:
country_stats_url = '/content/drive/My Drive/additional-kiva-snapshot/country_stats.csv'
country_stats_import = pd.read_csv(country_stats_url)
country_stats_import.dtypes

country_name                      object
country_code                      object
country_code3                     object
continent                         object
region                            object
population                         int64
population_below_poverty_line    float64
hdi                              float64
life_expectancy                  float64
expected_years_of_schooling      float64
mean_years_of_schooling          float64
gni                              float64
kiva_country_name                 object
dtype: object

In [ ]:
country_stats_import.head(5)

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


## Questions

####1.   **Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**

First of all, I cast the _lenders_ variable as an array

In [ ]:
loans_lenders_import['lenders'] = loans_lenders_import.lenders.apply(lambda x: x.split(','))
loans_lenders_import.head(2)

,loan_id,lenders
0,483693,"[muc888, sam4326, camaran3922, lachheb1865,..."
1,483738,"[muc888, nora3555, williammanashi, barbara5..."


Then, I can explode _lenders_ variable.

Please note: ".drop_duplicates()" is used to avoid duplicated lenders for load_in, if present in the original _lenders_ array

In [ ]:
loans_lenders = loans_lenders_import.explode('lenders').drop_duplicates()
loans_lenders.head(5)

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


####2.   **For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.**

I calculate _duration_ on the _loans_ dataframe, converting needed columns to datetime.

Please note: with _errors="coerce"_ option the system will set to NaN all values that cannot be converted.


In [ ]:
loans_import['planned_expiration_time']= pd.to_datetime(loans_import['planned_expiration_time'], format="%Y-%m-%d %H:%M:%S", errors="coerce") 
loans_import['disburse_time']= pd.to_datetime(loans_import['disburse_time'], format="%Y-%m-%d %H:%M:%S", errors="coerce") 

In [ ]:
loans_import['duration'] = loans_import['planned_expiration_time'] - loans_import['disburse_time']
loans_import.head(5)

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


####3.   **Find the lenders that have funded at least twice.**

In [ ]:
lender_foundings = loans_lenders.groupby('lenders').size().reset_index(name='foundings')
lender_foundings[lender_foundings['foundings'] > 2]

,lenders,foundings
0,000,39
1,00000,39
2,0002,68
4,0101craign0101,71
5,0132575,4
...,...,...
1639015,zx147,4
1639017,zyrah8525,4
1639018,zyrorl,3
1639020,zzaman,11


####4.   **For each country, compute how many loans have involved that country as borrowers.**

In [ ]:
country_loans = loans_import.groupby('country_code').size().reset_index(name='loans')
country_loans.head(10)

,country_code,loans
0,AF,2337
1,AL,3075
2,AM,13952
3,AZ,10172
4,BA,608
5,BF,3489
6,BG,296
7,BI,1727
8,BJ,5946
9,BO,25250


####5.   **For each country, compute the overall amount of money borrowed.**

In [ ]:
country_loans_amount = loans_import.groupby('country_code')['loan_amount'].agg('sum').reset_index(name='overall_founds')
country_loans_amount.head(5)

,country_code,overall_founds
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0


####6.   **Like the previous point, but expressed as a percentage of the overall amount lent.**

In [ ]:
country_loans_amount['overall_founds_perc'] = country_loans_amount.overall_founds / country_loans_amount.overall_founds.sum()

country_loans_amount.head(5)

,country_code,overall_founds,overall_founds_perc
0,AF,1967950.0,0.001666
1,AL,4307350.0,0.003646
2,AM,22950475.0,0.019427
3,AZ,14784625.0,0.012515
4,BA,477250.0,0.000404


####7.   **Like the three previous points, but split for each year (with respect to disburse time).**

In [ ]:
loans_import['disburse_year'] = pd.DatetimeIndex(loans_import['disburse_time']).year

In [ ]:
country_year_loans = loans_import.groupby(['country_code','disburse_year']).size().reset_index(name='loans')
country_year_loans_amount = loans_import.groupby(['country_code','disburse_year'])['loan_amount'].agg('sum').reset_index(name='overall_founds')
country_year_loans_amount['overall_founds_perc'] = country_year_loans_amount.overall_founds / country_year_loans_amount.overall_founds.sum()

In [ ]:
country_year_loans.head(5)

,country_code,disburse_year,loans
0,AF,2007.0,408
1,AF,2008.0,370
2,AF,2009.0,678
3,AF,2010.0,632
4,AF,2011.0,247


In [ ]:
country_year_loans_amount.head(5)

,country_code,disburse_year,overall_founds,overall_founds_perc
0,AF,2007.0,194975.0,0.000167
1,AF,2008.0,365375.0,0.000312
2,AF,2009.0,585125.0,0.000500
3,AF,2010.0,563350.0,0.000481
4,AF,2011.0,245125.0,0.000209


####8.   **For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.**

First of all, I need to assing to each lender/loan, the corresponding loan's details. So, I need to join the 2 dataset. To avoid run out of RAM, I reduce the number of variables selected on _loans_import_

In [ ]:
lender_loan_details = pd.merge(
                        loans_lenders,
                        loans_import[['loan_id','loan_amount']],
                        left_on=  ['loan_id'],
                        right_on= ['loan_id'], 
                        how = 'inner')

lender_loan_details.head(5)

,loan_id,lenders,loan_amount
0,483693,muc888,1225.0
1,483693,sam4326,1225.0
2,483693,camaran3922,1225.0
3,483693,lachheb1865,1225.0
4,483693,rebecca3499,1225.0


Then, it's possible to group the dataset to obtain the overall amount of money lent

In [ ]:
 lender_loan_details.groupby('lenders')['loan_amount'].agg('sum').reset_index(name='overall_money_lent')

,lenders,overall_money_lent
0,000,103750.0
1,00000,52250.0
2,0002,33200.0
3,00mike00,2000.0
4,0101craign0101,147050.0
...,...,...
1639021,zzanita,450.0
1639022,zzcyna7269,1550.0
1639023,zzinnia,950.0
1639024,zzmcfate,36400.0


####9.   **For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.**

First of all, I join the _lenders_ and the _loans_lenders_ dataset by lender name, removing lenders without a country code associated

In [ ]:
lenders_import_filtered = lenders_import[lenders_import.country_code.notnull()]

In [ ]:
lender_loan_country = pd.merge(
                        loans_lenders,
                        lenders_import_filtered[['permanent_name','country_code']],
                        left_on=  ['lenders'],
                        right_on= ['permanent_name'], 
                        how = 'inner')

lender_loan_country['lender_country'] = lender_loan_country['country_code']

lender_loan_country = lender_loan_country[['loan_id', 'lender_country']]

lender_loan_country.head(5)

,loan_id,lender_country
0,483693,US
1,483738,US
2,485000,US
3,486087,US
4,534428,US


Then, I join obtained dataset with the _loans_ dataset by loan ID

In [ ]:
lender_loan_country_full = pd.merge(
                        lender_loan_country.drop_duplicates(),
                        loans_import[['loan_id','loan_amount','country_code']],
                        left_on=  ['loan_id'],
                        right_on= ['loan_id'], 
                        how = 'inner')

In [ ]:
lender_loan_country_full['borrowed_country'] = lender_loan_country_full['country_code']

lender_loan_country_group = lender_loan_country_full.groupby(['lender_country','borrowed_country'])['loan_amount'].agg('sum').reset_index(name='overall_founds')
lender_loan_country_group.head(5)

,lender_country,borrowed_country,overall_founds
0,AD,EC,600.0
1,AD,ML,300.0
2,AD,PH,300.0
3,AE,AF,450.0
4,AE,AL,9775.0


Finally, I can group the obtained dataset by the 2 country columns to obtain requested information

In [ ]:
lender_loan_country_group_borrowers = lender_loan_country_group.groupby(['borrowed_country'])['overall_founds'].agg('sum').reset_index(name='amount_borrowed')
lender_loan_country_group_lenders = lender_loan_country_group.groupby(['lender_country'])['overall_founds'].agg('sum').reset_index(name='amount_lent')

lender_loan_country_group_join = pd.merge(
                        lender_loan_country_group_borrowers,
                        lender_loan_country_group_lenders,
                        left_on=  ['borrowed_country'],
                        right_on= ['lender_country'], 
                        how = 'inner')


lender_loan_country_group_join['country'] = lender_loan_country_group_join['borrowed_country']
lender_loan_country_group_join = lender_loan_country_group_join[['country','amount_borrowed','amount_lent']]

lender_loan_country_group_join['lent_borrowed_ratio'] = lender_loan_country_group_join['amount_borrowed']/lender_loan_country_group_join['amount_lent']
lender_loan_country_group_join['lent_borrowed_delta'] = lender_loan_country_group_join['amount_borrowed'] - lender_loan_country_group_join['amount_lent']

lender_loan_country_group_join.head(5)

,country,amount_borrowed,amount_lent,lent_borrowed_ratio,lent_borrowed_delta
0,AF,1738725.0,68075.0,25.541315,1670650.0
1,AL,3188775.0,16650.0,191.518018,3172125.0
2,AM,17855500.0,23575.0,757.391304,17831925.0
3,AZ,12580275.0,3200.0,3931.335938,12577075.0
4,BA,455825.0,89925.0,5.068946,365900.0


####10.   **Which country has the highest ratio between the difference computed at the previous point and the population?**

To evaluate this ratio, I've to join the previously created dataset with the _country_stats_ one

In [ ]:
lender_loan_country_group_stats = pd.merge(
                        lender_loan_country_group_join,
                        country_stats_import,
                        left_on=  ['country'],
                        right_on= ['country_code'], 
                        how = 'inner')

Then, I can compute the requested KPI

In [ ]:
lender_loan_country_group_stats1 = lender_loan_country_group_stats
lender_loan_country_group_stats1['population_ratio'] = lender_loan_country_group_stats1['lent_borrowed_delta']/lender_loan_country_group_stats1['population']
lender_loan_country_group_stats1 = lender_loan_country_group_stats1[['country','lent_borrowed_delta','population_ratio']]
lender_loan_country_group_stats1.head(5)

,country,lent_borrowed_delta,population_ratio
0,AF,1670650.0,0.047021
1,AL,3172125.0,1.082567
2,AM,17831925.0,6.085047
3,AZ,12577075.0,1.279772
4,BA,365900.0,0.104334


####11.   **Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?**

To evaluate it, we have to multiply the overall population number and the _population_below_poverty_line_ ratio information

In [ ]:
lender_loan_country_group_stats2 = lender_loan_country_group_stats
lender_loan_country_group_stats2['population_weighed'] = lender_loan_country_group_stats2['population_below_poverty_line'] * lender_loan_country_group_stats2['population']/100
lender_loan_country_group_stats2['population_weighed_ratio'] = lender_loan_country_group_stats2['lent_borrowed_delta']/lender_loan_country_group_stats2['population_weighed']
lender_loan_country_group_stats2 = lender_loan_country_group_stats2[['country','lent_borrowed_delta','population_ratio', 'population_weighed_ratio']]
lender_loan_country_group_stats2.head(5)

,country,lent_borrowed_delta,population_ratio,population_weighed_ratio
0,AF,1670650.0,0.047021,0.001313
1,AL,3172125.0,1.082567,0.075704
2,AM,17831925.0,6.085047,0.190158
3,AZ,12577075.0,1.279772,0.261178
4,BA,365900.0,0.104334,0.006066


####12.   **For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.**


Let's start defining a function that, given needed information (start date, end date and value) split it by years.

In [ ]:
def divide_value_by_period(row):  
  start_date = row['disburse_time'].tz_localize(None) # Es: 1 Dec 2013
  end_date = row['planned_expiration_time'].tz_localize(None) # Es: 31 Mar 2015
  value = row['loan_amount'] # Es: 400$

  # calculating the difference in years considewring leap years
  jumps = end_date.year - start_date.year # 2015 - 2013 = 2
  if jumps != 0:
    dayss = []
    starting_year = start_date.year # Iter 1: 2013
   
    for i in range(jumps): # Iterations: 0,1
      next_year = starting_year + 1 # Iter 1: 2014, Iter 2: 2015
      next_year_comp = datetime(next_year, 1, 1) # Iter 1: 1 Jan 2014, Iter 2: 1 Jan 2015
     
      # get the difference in days
      diff = (next_year_comp - start_date).days # Iter 1: 1 Jan 2014 - 1 Dec 2013 -> 31gg, Iter 2: 1 Jan 2015 - 1 Jan 2014 -> 365gg
      dayss.append(diff) # Iter 1: [31], Iter 2: [31,365]
      # re-assigning start and end dates
      starting_year = next_year_comp.year # Iter 1: 2014, Iter 2: 2015
      start_date = next_year_comp # Iter 1: 1 Jan 2014, Iter 3: 1 Jan 2015
 
    # Exiting from "for": dayss = [31,365] and start_date = 1 Jan 2015

    # adding the days between the end date and the first day of the last year
    dayss.append(((end_date - start_date).days) + 1) # 31 Mar 2015 - 1 Jan 2015 -> 90
    # dayss = [31,365,90]
 
    # calculating the portion of value each period gets
    if sum(dayss) > 0:
      # (31 * 400$) / (31+365+90)
      # (365 * 400$) / (31+365+90)
      # (90 * 400$) / (31+365+90)
      return [(x*value)/sum(dayss) for x in dayss]
    else:
      return value
  else:
    return value

Now, we can apply the funciton to the dataset, removing rows where one of the 2 dates are missing.
I also apply a check on overall duration, to remove issues (duration <= 0 days)

In [ ]:
time_loans = loans_import[loans_import.disburse_time.notnull() & loans_import.planned_expiration_time.notnull()]
time_loans = time_loans[time_loans.duration > pd.Timedelta(0,'D')]
time_loans['YearSplit'] = time_loans.apply(divide_value_by_period, axis=1)

time_loans.head(5)

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration,disburse_year,YearSplit
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06,2013.0,"[20.833333333333332, 104.16666666666667]"
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07,2013.0,"[45.833333333333336, 354.1666666666667]"
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05,2014.0,400
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02,2014.0,625
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02,2013.0,"[102.58620689655173, 322.41379310344826]"
